In [0]:
import pyspark

In [0]:
df=spark.sql("select * from titanic_csv")

In [0]:
df.printSchema()

root
-- PassengerId: string (nullable = true)
-- Survived: string (nullable = true)
-- Pclass: string (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: string (nullable = true)
-- SibSp: string (nullable = true)
-- Parch: string (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: string (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
from pyspark.sql.types import *

# Change column type
df = df.withColumn("PassengerId", df["PassengerId"].cast(IntegerType()))
df = df.withColumn("Survived", df["Survived"].cast(IntegerType()))
df = df.withColumn("Pclass", df["Pclass"].cast(IntegerType()))
df = df.withColumn("Age", df["Age"].cast(IntegerType()))
df = df.withColumn("SibSp", df["SibSp"].cast(IntegerType()))
df = df.withColumn("Parch", df["Parch"].cast(IntegerType()))
df = df.withColumn("Fare", df["Fare"].cast(FloatType()))

df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: integer (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: float (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
df.columns

Out[8]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
my_cols=df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare',
 'Embarked'])

In [0]:
final_data=my_cols.na.drop()

In [0]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,StringIndexer,OneHotEncoder)

In [0]:
gender_indexer=StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder=OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

Embarked_indexer=StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndex')
Embarked_encoder=OneHotEncoder(inputCol='EmbarkedIndex',outputCol='EmbarkedVec')

In [0]:
assembler=VectorAssembler(inputCols=['Pclass','SexVec','EmbarkedVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
pipeline=Pipeline(stages=[gender_indexer,Embarked_indexer,gender_encoder,Embarked_encoder,assembler,log_reg_titanic])

In [0]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [0]:
fit_model=pipeline.fit(train_data)

In [0]:
results=fit_model.transform(test_data)

In [0]:
results.show()

+--------+------+------+---+-----+-----+--------+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
Survived|Pclass| Sex|Age|SibSp|Parch| Fare|Embarked|SexIndex|EmbarkedIndex| SexVec| EmbarkedVec| features| rawPrediction| probability|prediction|
+--------+------+------+---+-----+-----+--------+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
 0| 1|female| 25| 1| 2| 151.55| S| 1.0| 0.0| (1,[],[])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0,...|[-2.8150084173383...|[0.05651851713032...| 1.0|
 0| 1| male| 18| 1| 0| 108.9| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[-0.6789111999315...|[0.33650435479686...| 1.0|
 0| 1| male| 22| 0| 0|135.6333| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[-0.8705147237422...|[0.29514720998634...| 1.0|
 0| 1| male| 33| 0| 0| 5.0| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[0.04345285985589...|[0.51086150600410...| 0.0|
 0| 1| male| 37| 1| 0| 53.1| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[0.47635175582583...|[0.61688602576214...| 0.0|
 0| 1| male| 38| 0| 0| 0.0| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|(8,[0,1,2,4],[1.0...|[0.26846702828344...|[0.56671652400630...| 0.0|
 0| 1| male| 38| 0| 1|153.4625| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[0.17799015879338...|[0.54438043533045...| 0.0|
 0| 1| male| 40| 0| 0| 0.0| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|(8,[0,1,2,4],[1.0...|[0.35560603935611...|[0.58797636549725...| 0.0|
 0| 1| male| 40| 0| 0| 27.7208| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[0.06841040035662...|[0.51709593321959...| 0.0|
 0| 1| male| 45| 0| 0| 28.5| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[0.53260371478619...|[0.63009018209513...| 0.0|
 0| 1| male| 45| 0| 0| 35.5| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[0.52257041774195...|[0.62774861924692...| 0.0|
 0| 1| male| 56| 0| 0| 26.55| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[1.01466326667028...|[0.73393176578777...| 0.0|
 0| 1| male| 61| 0| 0| 33.5| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[1.22254916262161...|[0.77251184354760...| 0.0|
 0| 1| male| 62| 0| 0| 26.55| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,1.0,0.0,...|[1.27608029988828...|[0.78178181809168...| 0.0|
 0| 1| male| 65| 0| 1| 61.9792| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[1.23802975534029...|[0.77522087848144...| 0.0|
 0| 2|female| 26| 1| 1| 26.0| S| 1.0| 0.0| (1,[],[])|(2,[0],[1.0])|[2.0,0.0,1.0,0.0,...|[-1.5582856862639...|[0.17389277752107...| 1.0|
 0| 2|female| 44| 1| 0| 26.0| S| 1.0| 0.0| (1,[],[])|(2,[0],[1.0])|[2.0,0.0,1.0,0.0,...|[-0.9035198294648...|[0.28832770875949...| 1.0|
 0| 2| male| 21| 1| 0| 11.5| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[2.0,1.0,1.0,0.0,...|[1.00155023095261...|[0.73136326333726...| 0.0|
 0| 2| male| 23| 0| 0| 15.0458| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[2.0,1.0,0.0,1.0,...|[0.50858035569256...|[0.62447361778277...| 0.0|
 0| 2| male| 25| 1| 2| 41.5792| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[2.0,1.0,0.0,1.0,...|[1.14422273820680...|[0.75845409497421...| 0.0|
+--------+------+------+---+-----+-----+--------+--------+--------+-------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [0]:
results.select(['Survived','prediction']).show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
AUC=my_eval.evaluate(results)
AUC

Out[30]: 0.7601677816714454